# Mixed language programming with SWIG

Simon Funke, Hans Petter Langtangen, Joakim Sundnes, Ola Skavhaug

Date: **Oct 5, 2016**

## The nature of Python vs. C

A Python variable can hold different objects:

```python
d = 3.2    # d holds a float
d = 'txt'  # d holds a string
d = Button(frame, text='push')  # instance of class Button
```

In C, C++ and Fortran, a variable is declared of a specific type:

```C
double d;
d = 4.2;
d = "some string";  /* illegal, compiler error */
```

This difference makes it quite complicated to call C, C++ or Fortran from Python

## Learn more about C?

[INF3331/INF4331 C tutorial](https://github.com/UiO-INF3331/student-resources-16/blob/master/lectures/07_introduction_to_c.pdf)

## Contents

* Calling C from Python - the hard way
* SWIG
    * Python-C
    * Python-C++
    * Numpy arrays and SWIG

# Calling C from Python - the hard way

## Calling C from Python

Suppose we have a C function (from an existing library or our own implementation):

```C
extern double hw1(double r1, double r2);
```

We want to call this from Python as

```python
from hw import hw1
r1 = 1.2
r2 = -1.2
s = hw1(r1, r2)
```

The Python variables `r1` and `r2` hold numbers (`float`). We need to extract these in the C code, convert to `double` variables, then call `hw1`, and finally convert the `double` result to a Python `float`.

All this conversion is done in *wrapper* code.

## Writing wrapper code

Python allows us to write wrapper functions in C that.
* Every object in Python is represented by a C struct `PyObject`
* Wrapper code converts between `PyObject` variables and plain C variables (from `PyObject` `r1` and `r2` to `double`, and `double` to `PyObject`).
* Once compiled it can be loaded as a Python module.

## Wrapper code for the `hw` function

```C
#include "Python.h"
#include "src/hw.h"

static PyObject *_wrap_hw1(PyObject *self, PyObject *args) {

  PyObject *resultobj;
  double r1, r2, result;

  PyArg_ParseTuple(args, (char *)"dd:hw1", &r1, &r2);

  hw1(r1, r2, &result);

  resultobj = PyFloat_FromDouble(result);
  return resultobj;
}
```

*Note*: The first function argument (`self`) points to the module object for module-level functions; for a method it points to the object instance.

The full example can be found [here](https://github.com/UiO-INF3331/code-snippets-16/tree/master/mixed/c-api).

## Using the wrapper

* The wrapper function and *hw1* must be compiled and linked to a shared library file.
* This file can be loaded in Python as module.

$->$ Demo

* Direct calls through wrapper code enables efficient data transfer: large arrays can be sent by pointers
* Such modules written in other languages are called **extension modules**.

## Writing wrapper code is tedious

A wrapper function is needed for each C function we want to call from Python

Let's automated it!

We shall use SWIG (for C/C++)

# Simplified Wrapper and Interface Generation (SWIG)

## More info

* [SWIG manual](http://www.swig.org/Doc1.3/Contents.html)
* Code snippets, swig example: https://github.com/UiO-INF3331/code-snippets-16/tree/master/mixed/swig

## Scientific Hello World example

Consider this Scientific Hello World module (`hw.py`):

In [ ]:
import math

def hw1(r1, r2):                                                                                                                                                                               
    s = math.sin(r1 + r2)                                                                                                                                                                      
    return s    

def hw2(r1, r2):
    s = math.sin(r1 + r2)
    print('sin({:.2f}+{:.2f})={:.2f}'.format(r1, r2, s))

Usage:

In [ ]:
from hw import hw1
s = hw1(1.0, 0)
hw2(1.0, 0)

We want to implement the module in C and C++, and use it as if it was a pure Python module

## Using SWIG to wrap C and C++

* Wrappers to C and C++ codes is automatically generated by SWIG
* SWIG is more complicated to use than Cython:
    1. First make a SWIG interface file
    2. Then run SWIG to generate wrapper code
    3. Finally compile and link the C code and the wrapper code

# C implementation of `hw1` and `hw2`

Header file:

```C
// file: src/hw.h
void hw1(double r1, double r2, double *s);
void hw2(double r1, double r2);
```    

Source file:

```C
// file: src/hw.c
#include <stdio.h>
#include <math.h>

#include "hw.h"

void hw1(double r1, double r2, double *s)
{
    *s = sin(r1 + r2);
}

void hw2(double r1, double r2)
{
    double s;
    s = sin(r1 + r2);
    printf("sin(%f+%f)=%f\n", r1, r2, s);
}
```    

## SWIG interface file

The interface file contains C preprocessor directives and special SWIG directives:

```C
/* file: hw.i */
%module hw
%{
/* Everything in this block will be copied in the wrapper file. We include the C header file necessary to compile the interface
*/
#include "src/hw.h"
%}

/* list functions to be interfaced: */
void hw1(double r1, double r2, double *s);
void hw2(double r1, double r2);
```

## Generating the module

Run SWIG (preferably in a subdirectory):

```bash
swig -python -Isrc hw.i
```

SWIG generates wrapper code in
```bash
hw_wrap.c
```

Compile and link a shared library module:

```bash
gcc -Isrc -fPIC -I/usr/include/python3.5m -I/usr/include/x86_64-linux-gnu/python3.5m -lpython3.5m -c src/hw.c hw_wrap.c
gcc -shared -fPIC -o _hw.so hw.o hw_wrap.o
```

**Note:** the underscore prefix in `_hw.so` is required!

## A build script

* Make a script to automate the compile+link process
* Can use `pkg-config` tool to extract where `Python.h` resides and to use the correct compiler flags.

```bash
# file: build.sh
swig -python -Isrc hw.i

gcc -Isrc -fPIC $(pkg-config --cflags --libs python3) -c src/hw.c hw_wrap.c
gcc -shared -fPIC -o _hw.so hw.o hw_wrap.o

python -c "import hw" # test
```

The compiled module consists of two files: `hw.py` (which loads), `_hw.so`

The full example and the build script is found [here](https://github.com/UiO-INF3331/code-snippets-16/tree/master/mixed/swig/c)

## Building modules with Distutils (1)

* We can use Distutils to compile and link extension modules reliably
* First write a script `setup.py`:

In [ ]:
from distutils.core import setup, Extension

name = "hw"      # name of the module
version = "1.0"  # the module's version number

setup(name=name, version=version,
      ext_modules=[Extension(name='_hw', 
      # SWIG requires an underscore as a prefix for the module name
             sources=["hw.i", "src/hw.c"],
             include_dirs=['src'])
    ])

## Building modules with Distutils (2)

Now compile and install the module

```bash
# compile module
python setup.py build_ext 
# install in the current directory
python setup.py install --install-platlib=. 
```

and test with:

```bash
python -c 'import hw'     # test module
```

*Note*:
* You can install resulting module files in any directory (make sure that `PYHONPATH` points to it)
* Use Distutils for professional distribution!

## Testing our implementation

Recall:

```C
void hw1(double r1, double r2, double *s)
{
  *s = sin(r1 + r2);
}
```

Test:

```ipython
from hw import hw1
r1 = 1;  r2 = -1;  s = 10
hw1(r1, r2, s)
>>> TypeError: in method 'hw1', argument 3 of type 'double *'
```

**Upps!**

## Specifying input/output arguments

We need to adjust the SWIG interface file:

```C
/* typemaps.i allows input and output pointer arguments to be specified using the names INPUT, OUTPUT, or INOUT */
%include "typemaps.i"

void   hw1(double r1, double r2, double *OUTPUT);
```

Now the usage from Python is

```python
s = hw1(r1, r2)
```

## Other tools

* [Boost](http://www.boost.org/doc/libs/1_62_0/libs/python/doc/html/index.html).Python: tool for wrapping C++ libraries
* [PyCXX](http://cxx.sourceforge.net/): C++ facilitates to simplify interfacing C++ with Python
* [Instant](https://bitbucket.org/fenics-project/instant): simple tools for inlining C and C++ code in Python scripts
* [Cython](http://cython.org/) C-Extensions for Python

Note: SWIG can generate interfaces to most scripting languages (Perl, Ruby, Tcl, Java, Guile, Mzscheme, ...)

## Which tool should I use??
<center>![Mixed Programming Python tools](python_integration_tools.svg "Python")</center>

## Integrating Python with C++

* SWIG supports C++

* The only difference is when we run SWIG (`-c++` option):

```bash
swig -python -c++ -Isrc helloworld.i
# generates wrapper code in helloworld_wrap.cxx
```

Use a C++ compiler to compile and link:

```bash
g++ -Isrc -fPIC -c $(pkg-config --cflags --libs python3) src/HelloWorld.cpp helloworld_wrap.cxx
g++ -shared -o _helloworld.so HelloWorld.o helloworld_wrap.o
```

## Interfacing C++ functions (1)

This is like interfacing C functions, except that pointers are usual replaced by references (less powerfull but safer)

```C++
// C style
void hw3(double r1, double r2, double *s)  
{ *s = sin(r1 + r2); }

// C++ style
void hw4(double r1, double r2, double& s)  
{ s = sin(r1 + r2); }
```

## Interfacing C++ functions (2)

Interface file (`HelloWorld.i`):

```C++
%module helloworld
%{
#include "src/HelloWorld.h"
%}


%include "typemaps.i"

/* The apply directive is used to change the datatype to OUTPUTs */
%apply double *OUTPUT { double* s }
%apply double *OUTPUT { double& s }
%include "src/HelloWorld.h" 
```

That's it!

# Interfacing C++ classes

* C++ classes add more to the SWIG integration story
* Consider a class version of our Hello World module:

```C++
/* file: src/HelloWorld.h */
class HelloWorld
{
 protected:
  double r1, r2, s;
  void compute();    // compute s=sin(r1+r2)
 public:
  HelloWorld();
  ~HelloWorld();

  void set(double r1, double r2);
  double get() const { return s; }
  void message(std::ostream& out) const;
};
```

* Goal: use this class as a Python class

## Function bodies and usage

Function bodies:

```C++
/* file: src/HelloWorld.cpp */
void HelloWorld::set(double r1_, double r2_)
{
  r1 = r1_;  r2 = r2_;
  compute();  // compute s
}
void HelloWorld::compute()
{ s = sin(r1 + r2); }
```

etc.

**Usage**:

```C++
HelloWorld hw;
hw.set(r1, r2);
hw.message(std::cout);  // write "Hello, World!" message
```

## Adding a subclass

To illustrate how to handle class hierarchies, we add a subclass:

```C++
/* file: src/HelloWorld2.h */
class HelloWorld2 : public HelloWorld
{
 public:
  void gets(double& s_) const;
};
```

and

```C++
/* file: src/HelloWorld2.cpp */
void HelloWorld2::gets(double& s_) const { s_ = s; }
```

That is, we have a function with an output argument

**Note**: `gets` should return the value when called from Python

## SWIG interface file

```C++
/* file: helloworld.i */
%module helloworld
%{
/* include C++ header files necessary to compile the interface */
#include "HelloWorld.h"
#include "HelloWorld2.h"
%}

%include "src/HelloWorld.h"

%include "typemaps.i"
%apply double* OUTPUT { double& s }
%include "HelloWorld2.h"
```

## Adding a class method

* SWIG allows us to add class methods

* Calling `message` with standard output (`std::cout`) is tricky from Python so we add a `print` method for printing to `std.output`

* `print` coincides with Python's keyword `print` so we follow the convention of adding an underscore:

```C++
%extend HelloWorld {
    void print_() { self->message(std::cout); }
}
```

This is basically C++ syntax, but `self` is used instead of `this` and `%extend HelloWorld` is a SWIG directive

## Make and compile extension module

We can create a `setup.py` file for compiling and installing the module:

In [ ]:
from distutils.core import setup, Extension

name = "helloworld"    # name of the module
version = "1.0"        # the module's version number

setup(name=name, version=version,
      # distutils detects .i files and compiles them automatically
      ext_modules=[Extension(name='_helloworld', # SWIG requires _ as a prefix for the module name
                             sources=["helloworld.i", 
                                      "src/HelloWorld.cpp", 
                                      "src/HelloWorld2.cpp"],
                             include_dirs=['src'],
                             swig_opts=["-c++"])
    ])

Build and compile with:
```bash
python setup.py build_ext
python setup.py install --install-platlib=.
```
The full example is available [here](https://github.com/UiO-INF3331/code-snippets-16).

## Using the module

In [ ]:
from helloworld import HelloWorld, HelloWorld2

hw = HelloWorld()     # make class instance
hw.set(r1, r2)        # call instance method
s = hw.get()
hw.print_()

In [ ]:
hw2 = HelloWorld2()   # make subclass instance
hw2.set(r1, r2)
s = hw2.gets()        # original output arg. is now return value

## Remark

* It looks like if the C++ class hierarchy is mirrored in Python

* Actually, SWIG wraps a `function` interface to any class:

In [ ]:
import _helloworld   # use _helloworld.so directly
hw = _helloworld.new_HelloWorld()
_helloworld.HelloWorld_set(hw, r1, r2)

SWIG makes a proxy class in `helloworld.py`, mirroring the original C++ class:

In [ ]:
import helloworld    # use helloworld.py interface to _helloworld.so
hw = helloworld.HelloWorld()
hw.set(r1, r2)       # calls _hw.HelloWorld_set(r1, r2)

*Note:* The proxy class introduces some overhead